# Import

In [84]:
import pandas as pd
import os

In [85]:
# Get the current file's directory
current_dir = ocurrent_dir = os.getcwd()

project_root = os.path.dirname(current_dir)

data_dir = os.path.join(project_root, 'data')
customers_file = os.path.join(data_dir, 'customers_raw.csv')
complaints_file = os.path.join(data_dir, 'complaints_raw.csv')

customers_df = pd.read_csv(customers_file)
complaints_df = pd.read_csv(complaints_file)

# Clean

In [86]:
# #inspect unique values and missing values

# for col in customers_df.columns:
#     print('----\n', customers_df[col].value_counts(), '\n\n', customers_df[col].isna().sum())

In [87]:
# New customers have positive monthly charges but no total charges.
customers_df.loc[customers_df['TotalCharges'] == ' ', 'TotalCharges'] = customers_df.loc[customers_df['TotalCharges'] == ' ', 'MonthlyCharges']

In [88]:
## Customer's data

# Column names and variable types
customers_df = customers_df.rename(columns={'customerID': 'CustomerId', 'tenure':'Tenure', 'gender':'Gender'})
customers_df['CustomerId'] = customers_df['CustomerId'].astype(str)
customers_df['Gender'] = customers_df['Gender'].astype(str)
customers_df['SeniorCitizen'] = customers_df['SeniorCitizen'].astype(int)
customers_df['Dependents'] = customers_df['Dependents'].astype(str).map({'No':0, 'Yes':1})
customers_df['Tenure'] = customers_df['Tenure'].astype(int)
customers_df['PhoneService'] = customers_df['PhoneService'].astype(str).map({'No':0, 'Yes':1})
customers_df['MultipleLines'] = customers_df['MultipleLines'].astype('category')
customers_df['InternetService'] = customers_df['InternetService'].astype('category')
customers_df['OnlineSecurity'] = customers_df['OnlineSecurity'].astype('category')
customers_df['OnlineBackup'] = customers_df['OnlineBackup'].astype('category')
customers_df['DeviceProtection'] = customers_df['DeviceProtection'].astype('category')
customers_df['TechSupport'] = customers_df['TechSupport'].astype('category')
customers_df['StreamingTV'] = customers_df['StreamingTV'].astype('category')
customers_df['StreamingMovies'] = customers_df['StreamingMovies'].astype('category')
customers_df['Contract'] = customers_df['Contract'].astype('category')
customers_df['ContractLengthMonths'] = customers_df['Contract'].astype(str).map({'Month-to-month':1, 'One year':12, 'Two year':24})
customers_df = customers_df.drop(columns=['Contract'])
customers_df['PaperlessBilling'] = customers_df['PaperlessBilling'].astype(str).map({'No':0, 'Yes':1})
customers_df['PaymentMethod'] = customers_df['PaymentMethod'].astype('category')
customers_df['MonthlyCharges'] = customers_df['MonthlyCharges'].astype(float)
customers_df['TotalCharges'] = customers_df['TotalCharges'].astype(float)
customers_df['Churn'] = customers_df['Churn'].astype(str).map({'No':0, 'Yes':1})

# convert to one hot encoding
categorical_columns = customers_df.select_dtypes(include=['category']).columns

# Create one-hot encoding for each categorical column
for column in categorical_columns:
    # Create dummy variables and add prefix with column name
    dummies = pd.get_dummies(customers_df[column], prefix=column)
    
    # Add dummy columns to dataframe
    customers_df = pd.concat([customers_df, dummies], axis=1)
    
    # Drop the original categorical column
    customers_df = customers_df.drop(column, axis=1)

In [89]:
## Complaints data

# Column names and variable types
complaints_df = complaints_df.rename(columns={'customerID': 'CustomerId', 'complaint':'Complaint', 'complaint_number':'ComplaintNumber'})
complaints_df['CustomerId'] = complaints_df['CustomerId'].astype(str)
complaints_df['Complaint'] = complaints_df['Complaint'].astype(str)
complaints_df['ComplaintNumber'] = complaints_df['ComplaintNumber'].astype(int)

# Eng. features

check if recording date of user is before or after monthly charge

In [90]:
## Customer df

# 1. ChangedPlan
customers_df['ChangedPlan'] = 0
expected_total = customers_df['Tenure'] * customers_df['MonthlyCharges']
customers_df.loc[customers_df['TotalCharges'] != expected_total, 'ChangedPlan'] = 1

# 2. ChangedPlanPositive
customers_df['ChangedPlanPositive'] = 0
customers_df.loc[customers_df['TotalCharges'] < expected_total, 'ChangedPlanPositive'] = 1

# 3. ChangedPlanNegative
customers_df['ChangedPlanNegative'] = 0
customers_df.loc[customers_df['TotalCharges'] > expected_total, 'ChangedPlanNegative'] = 1

# 4. InternetServicesDensity
# First, get all service columns
internet_services_columns = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                             'TechSupport', 'StreamingTV', 'StreamingMovies']

# Count services that are not "No" and divide by 7
# Define the internet services columns after one-hot encoding
internet_services_binary = [col for col in customers_df.columns if any(service in col for service in [
    'OnlineSecurity_Yes',
    'OnlineBackup_Yes',
    'DeviceProtection_Yes',
    'TechSupport_Yes',
    'StreamingTV_Yes',
    'StreamingMovies_Yes'
])]

# Calculate the density using the binary columns
customers_df['InternetServicesDensity'] = customers_df[internet_services_binary].sum(axis=1) / len(internet_services_binary)

# # 6. ContractLifecycle
# customers_df['ContractLifecycle'] = customers_df.apply(
#     lambda row: (row['Tenure'] % row['ContractLengthMonths']) / row['ContractLengthMonths'], 
#     axis=1
# )

In [91]:
from textblob import TextBlob
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Text preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    # Convert to lowercase
    text = text.lower()

    # Handle contractions
    text = text.replace("'m", " am")
    text = text.replace("n't", " not")
    text = text.replace("'re", " are")
    text = text.replace("'s", " is")
    text = text.replace("'ll", " will")
    text = text.replace("'ve", " have")
    text = text.replace("'d", " would")

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

# Function to get key terms by POS tag for churned customers
def get_key_terms_by_pos(complaints_df, customers_df, pos_tag_type, top_n=10):
    # Merge complaints with customer data
    merged_df = pd.merge(complaints_df, customers_df[['CustomerId', 'Churn']], on='CustomerId', how='left')
    
    # Get complaints from churned customers
    churned_complaints = merged_df[merged_df['Churn'] == 0]['Complaint'].dropna()
    
    # Process all complaints
    processed_complaints = [preprocess_text(text) for text in churned_complaints]
    
    # Get all words with specified POS tag
    key_terms = []
    for text in processed_complaints:
        # POS tag the text
        tagged = pos_tag(word_tokenize(text))
        # Extract words with matching POS tag
        if pos_tag_type == 'NOUN':
            pos_filter = ['NN', 'NNS', 'NNP', 'NNPS']
        elif pos_tag_type == 'VERB':
            pos_filter = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
        elif pos_tag_type == 'ADJ':
            pos_filter = ['JJ', 'JJR', 'JJS']
        
        key_terms.extend([word.lower() for word, tag in tagged if tag in pos_filter])
    
    # Get most common terms
    return [word for word, _ in Counter(key_terms).most_common(top_n)]

def count_terms(text, terms):
    if not text:
        return 0
    words = text.split()
    return sum(words.count(term) for term in terms)


# Create the features

# 0. ProcessedComplaint
complaints_df['ProcessedComplaint'] = complaints_df['Complaint'].fillna('').apply(preprocess_text)

# 1. ComplaintLength
complaints_df['ComplaintLength'] = complaints_df['Complaint'].fillna('').str.len()

# Get key terms for churned customers
key_nouns = get_key_terms_by_pos(complaints_df, customers_df, 'NOUN')
key_verbs = get_key_terms_by_pos(complaints_df, customers_df, 'VERB')
key_adjectives = get_key_terms_by_pos(complaints_df, customers_df, 'ADJ')

# Process complaints
processed_complaints = complaints_df['ProcessedComplaint']


# 2. Sentiment
complaints_df['Sentiment'] = complaints_df['Complaint'].fillna('').apply(
    lambda x: TextBlob(str(x)).sentiment.polarity
)

# 3. KeyVerbsCount
complaints_df['KeyVerbsCount'] = processed_complaints.apply(
    lambda x: count_terms(x, key_verbs)
)

# 4. KeyNounsCount
complaints_df['KeyNounsCount'] = processed_complaints.apply(
    lambda x: count_terms(x, key_nouns)
)

# 5. KeyAdjectivesCount
complaints_df['KeyAdjectivesCount'] = processed_complaints.apply(
    lambda x: count_terms(x, key_adjectives)
)

[nltk_data] Downloading package punkt to /Users/marco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/marco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/marco/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [92]:
# anger df for anger score calculation
anger_df = complaints_df.copy()
max_complaint_number = anger_df.groupby('CustomerId')['ComplaintNumber'].aggregate('max')
anger_df = anger_df.merge(max_complaint_number, on=['CustomerId', 'ComplaintNumber'], how='inner')[['CustomerId', 'ComplaintNumber', 'Complaint']]

## pipeline to compute anger score with gen ai
# ..
# -> anger_df_result

In [93]:
# compacting complaints into one column to have one row per customer
compacted_complanits_df = anger_df.groupby('CustomerId').aggregate({'Complaint': ' '.join}).reset_index()
# compacted_complanits_df['AngerScore'] = anger_df_result['AngerScore'].values  


# Merged

In [94]:
merdged_df = pd.merge(customers_df, compacted_complanits_df, on='CustomerId', how='left')
merdged_df['HasComplaint'] = merdged_df['Complaint'].notna().astype(int)

question: how to merge complaints with customer data when a user has more than one complaint?
solution: compute anger score for the last complaint, append all complaints into into one,

# Save CSV

In [99]:
complaints_df.to_csv('/Users/marco/Documents/python_projects/churn_bcgx/data/complaints_processed.csv', index=False)
customers_df.to_csv('/Users/marco/Documents/python_projects/churn_bcgx/data/customers_processed.csv', index=False)
merdged_df.to_csv('/Users/marco/Documents/python_projects/churn_bcgx/data/customers_complaints_merdged.csv', index=False)
anger_df.to_csv('/Users/marco/Documents/python_projects/churn_bcgx/data/complaitns_for_anger.csv', index=False)